<a href="https://colab.research.google.com/github/kaifkh20/nn_z_to_h/blob/main/nano_gpt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/refs/heads/master/data/tinyshakespeare/input.txt

--2025-05-10 08:04:17--  https://raw.githubusercontent.com/karpathy/char-rnn/refs/heads/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.01s   

2025-05-10 08:04:17 (97.7 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [2]:
import torch
import torch.nn as nn
from torch.nn import functional as F

In [101]:
#hyperparameters

batch_size = 64
block_size = 256
max_iters = 5000
eval_interval = 200
learning_rate = 3e-4
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 384
n_head = 6
n_layers = 6
dropout = 0.2


In [85]:
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [86]:
print("length of dataset",len(text))

length of dataset 1115394


In [87]:
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [88]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [89]:
stoi = {ch:i for i,ch in enumerate(chars)}
itos = {i:ch for i,ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

print(encode("hii there"))
print(decode(encode("hii there")))

[46, 47, 47, 1, 58, 46, 43, 56, 43]
hii there


In [90]:
#store the data set encoded in a tensor
import torch
data = torch.tensor(encode(text),dtype = torch.long)
print(data.shape,data.dtype)
print(data[:1000])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

In [91]:
#split 90-train 10-val
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [92]:
#sampling data of each block_size from random batch

@torch.no_grad()
def estimate_loss():
  out = {}
  m.eval()
  for split in ['train','val']:
    losses = torch.zeros(block_size)
    for k in range(block_size):
      X,Y = get_batch(split)
      logits,loss = m(X,Y)
      losses[k] = loss.item()
    out[split] = losses.mean()
  m.train()
  return out

In [93]:
train_data[:block_size+1].data
# decode(train_data[:block_size+1])

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
        47, 59, 57,  1, 47, 57,  1, 41, 

In [94]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} the target: {target}")

when input is tensor([18]) the target: 47
when input is tensor([18, 47]) the target: 56
when input is tensor([18, 47, 56]) the target: 57
when input is tensor([18, 47, 56, 57]) the target: 58
when input is tensor([18, 47, 56, 57, 58]) the target: 1
when input is tensor([18, 47, 56, 57, 58,  1]) the target: 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) the target: 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target: 58
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47, 58]) the target: 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47]) the target: 64
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64]) the target: 43
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43]) the target: 52
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52]) the target: 10
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10]) the target: 0
when input is tensor([18, 47, 56, 57, 58,  1, 15, 

This trains in batches instead of training one by one of block size.

Instead of training like this:

[seq1] → loss → update weights  
[seq2] → loss → update weights  
[seq3] → ...

You train like this:

[seq1, seq2, seq3, seq4] → compute all losses together → average → update weights once


In [95]:
torch.manual_seed(1337)

def get_batch(split):
  #generate a small batch of input x and target y
  ix = torch.randint(len(data)-block_size,(batch_size,)) #pick starting point for each from ex if 1000->data then pick a rand no till 992
  x = torch.stack([data[i:i+block_size]for i in ix]) #for each index i, take a slice of 8 tokens starting at position i from data.
  y = torch.stack([data[i+1:i+block_size+1]for i in ix]) # For the same indices, take the next 8 tokens, shifted by 1:
  return x,y

xb,yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print("=====")

for b in range(batch_size): #batch dim
  for t in range(block_size): #block size
    context = xb[b,:t+1]
    target = yb[b,t]
    print(f"when input is {context.tolist()} the target is {target}")

Streaming output truncated to the last 5000 lines.
when input is [46, 53, 61, 1, 58, 53, 1, 61, 43, 50, 41, 53, 51, 43, 1, 59, 57, 1, 58, 53, 1, 58, 53, 61, 52, 8, 0, 0, 14, 13, 28, 32, 21, 31, 32, 13, 10, 0, 19, 43, 52, 58, 50, 43, 51, 43, 52, 6, 1, 47, 51, 54, 53, 56, 58, 59, 52, 43, 1, 51, 43, 1, 52, 53, 1, 44, 39, 56, 58, 46, 43, 56, 6, 0, 18, 53, 56, 1, 46, 53, 61, 1, 21, 1, 44, 47, 56, 51, 50, 63, 1, 39, 51, 1, 56, 43, 57, 53, 50, 60, 43, 42, 1, 63, 53, 59, 1, 49, 52, 53, 61, 11, 0, 32, 46, 39, 58, 1, 47, 57, 6] the target is 1
when input is [46, 53, 61, 1, 58, 53, 1, 61, 43, 50, 41, 53, 51, 43, 1, 59, 57, 1, 58, 53, 1, 58, 53, 61, 52, 8, 0, 0, 14, 13, 28, 32, 21, 31, 32, 13, 10, 0, 19, 43, 52, 58, 50, 43, 51, 43, 52, 6, 1, 47, 51, 54, 53, 56, 58, 59, 52, 43, 1, 51, 43, 1, 52, 53, 1, 44, 39, 56, 58, 46, 43, 56, 6, 0, 18, 53, 56, 1, 46, 53, 61, 1, 21, 1, 44, 47, 56, 51, 50, 63, 1, 39, 51, 1, 56, 43, 57, 53, 50, 60, 43, 42, 1, 63, 53, 59, 1, 49, 52, 53, 61, 11, 0, 32, 46, 39, 58, 1

In [96]:
class Head(nn.Module):
  #""single head of self-attention""

  def __init__(self,head_size):
    super().__init__()
    self.key = nn.Linear(n_embd,head_size,bias=False)
    self.query = nn.Linear(n_embd,head_size,bias=False)
    self.value = nn.Linear(n_embd,head_size,bias=False)
    self.register_buffer('tril',torch.tril(torch.ones(block_size,block_size)))
    self.dropout = nn.Dropout(dropout)
  def forward(self,x):
    B,T,C = x.shape
    k = self.key(x)
    q = self.query(x)

    #COMPUTE ATTENTION SCORES"

    wei = q @ k.transpose(-2,-1) * C**-0.5
    wei = wei.masked_fill(self.tril[:T,:T]==0,float('-inf'))
    wei = F.softmax(wei,dim=-1)
    wei = self.dropout(wei)
    # weighted agggregation of the values
    v = self.value(x)
    out = wei @ v
    return out

self.proj is for Projection Layer, it's used to project from
one space to another.

Each head's output is concats into output, and at first glance this might feel redundant, but it's not the output from each head if not projected works in their own subspace and don't mix.

We add the projection layer to make the combination of attention heads learnable — so the model can learn from its own multi-headed outputs instead of just passing them through unchanged.

In [97]:
class MultiHeadAttention(nn.Module):
  """multiple heads of self-attention in parallel"""

  def __init__(self,num_heads,head_size):
    super().__init__()
    self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
    self.proj = nn.Linear(n_embd,n_embd) #projection layer
    self.dropout = nn.Dropout(dropout)

  def forward(self,x):
    out = torch.cat([h(x) for h in self.heads],dim=-1)
    out = self.proj(out)
    return out


This is a 2-layer FeedForward network inside the transformer block, and it's a common pattern. The hidden layer expands the embedding size by a factor of 4 (i.e., 4 * n_embd) before projecting it back down to the original size.
✅ Reason: More Capacity for Nonlinear Modeling

    Increasing the dimension temporarily gives the model more representational power.

    The middle layer (4 * n_embd) allows the model to learn richer, more expressive features before projecting back down.

 Dropout is a regularization technique used during training to prevent overfitting.

    During training, Dropout(p) randomly sets a fraction p of the input neurons to zero on each forward pass.

    This forces the model to not rely too much on any single neuron, improving generalization.

In [98]:
class FeedForward(nn.Module):
  '''simple linear layer followed by non linearity'''
  def __init__(self,n_embd):
    super().__init__()
    self.net = nn.Sequential(
        nn.Linear(n_embd,n_head*n_embd),
        nn.ReLU(),
        nn.Linear(n_head*n_embd,n_embd), #projection layer
        nn.Dropout(dropout),
    )
  def forward(self,x):
    return self.net(x)

The x + self.sa(x) and x + self.ffwd(x) lines implement residual connections (also known as skip connections) in the transformer block architecture.

Each layer applies a transformation, usually a matrix multiply + activation (like ReLU or tanh). Now, during backpropagation, the gradient must pass through each layer in reverse.

But here’s the problem:

    If each layer has a derivative < 1 (like sigmoid or tanh), the gradients get smaller and smaller as they propagate backward.

    This is the vanishing gradient problem. Eventually, early layers stop learning.

Each block is now computing:
output=x+f(x)
output=x+f(x)

Where f(x)f(x) is some transformation (like attention or feedforward).

    Let’s simplify and assume the output of a block is:
    y=x+f(x)
    y=x+f(x)

    During backpropagation:
    ∂Loss∂x=∂Loss∂y⋅(1+∂f(x)∂x)
    ∂x∂Loss​=∂y∂Loss​⋅(1+∂x∂f(x)​)

In [75]:
class Block(nn.Module):
  def __init__(self,n_embd,n_head):
    super().__init__()
    head_size = n_embd//n_head
    self.sa = MultiHeadAttention(n_head,head_size)
    self.ffwd = FeedForward(n_embd)
    self.ln1 = nn.LayerNorm(n_embd)
    self.ln2 = nn.LayerNorm(n_embd)

  def forward(self,x):
    x = x+ self.sa(self.ln1(x)) #residual connections
    x = x+ self.ffwd(self.ln2(x))
    return x

In [102]:
torch.manual_seed(1337)
class BigramLanguageModel(nn.Module):
  def __init__(self,vocab_size):
    super().__init__()
    self.token_embedding_table = nn.Embedding(vocab_size,n_embd)
    self.position_embedding_table = nn.Embedding(block_size,n_embd)
    # self.sa_heads = MultiHeadAttention(4,n_embd//4)
    # self.ffwd = FeedForward(n_embd)
    # self.blocks = nn.Sequential(
    #     Block(n_embd,n_head=4),
    #     Block(n_embd,n_head=4),
    #     Block(n_embd,n_head=4),
    #     nn.LayerNorm(n_embd)
    # )

    self.blocks = nn.Sequential(*[Block(n_embd,n_head) for _ in range(n_layers)])
    self.ln_f = nn.LayerNorm(n_embd) #final layer norm
    self.lm_head = nn.Linear(n_embd,vocab_size)

  def forward(self,idx,targets=None):
    B,T = idx.shape
    tok_embd = self.token_embedding_table(idx) #B,T,C
    #We use a position embedding table to inject position information into the model
    #in which order does the word error
    #tok_embd only represent each word or token in n-d space without any order
    #pos_embd is helpful in enforcing order information
    pos_embd = self.position_embedding_table(torch.arange(T)) #T,C
    # print(pos_embd.shape,tok_embd.shape)
    x = tok_embd + pos_embd #B,T,C
    x = self.blocks(x)
    x = self.ln_f(x)
    logits = self.lm_head(x) #B,T,vocab_size

    if targets is None:
      loss = None
    else:

      B,T,C = logits.shape
      logits = logits.view(B*T,C)
      targets = targets.view(B*T)

      loss = F.cross_entropy(logits,targets)

    return logits,loss

  def generate(self,idx,max_new_tokens):
    for _ in range(max_new_tokens):

      #crop the idx till block_size since we need to feed it into attention head
      idx_crop = idx[:,-block_size:]

      logits,loss = self(idx_crop)
      # Only keep the logits for the last token in the sequence
      # Shape goes from [batch_size, seq_len, vocab_size] → [batch_size, vocab_size]
    # This is because we only care about predicting the next token in the sequence
      logits = logits[:,-1,:]
      probs = F.softmax(logits,dim=-1)
      idx_next = torch.multinomial(probs,num_samples=1)
      idx = torch.cat((idx,idx_next),dim=1)

    return idx

m = BigramLanguageModel(vocab_size)
logits,loss = m(xb,yb) # 4 batches each batch have block_size=8 each having prob=65


print(logits.shape)
print(loss)
print(decode(m.generate(torch.zeros((1,1),dtype=torch.long),max_new_tokens=100)[0].tolist()))
# print(out)


torch.Size([16384, 65])
tensor(4.2817, grad_fn=<NllLossBackward0>)

BUUvv:-AyDAn TFj&cO;KNR?B'oPOU!k&'f$wqg'GKBS,rWB!.avueD-Qd,ywP'fY&CBr e,d'mFrt&n,bsiWawS,s$yr &RkSqZ


In [104]:
optimizer = torch.optim.AdamW(m.parameters(),lr=learning_rate)

In [ ]:
for steps in range(max_iters):

  if steps % eval_interval  == 0:
    losses = estimate_loss()
    print(f"step {steps}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

  xb,yb = get_batch('train')
  logits,loss = m(xb,yb)
  optimizer.zero_grad(set_to_none=True)
  loss.backward()
  optimizer.step()

print(loss.item())

In [ ]:
 print(decode(m.generate(torch.zeros((1,1),dtype=torch.long),max_new_tokens=500)[0].tolist()))

Mathematical trick in self-attention.

for every t, we need to calculate the mean upto t.
{because what we need is that every token t interacts with only its previous context tokens and not the future}

The formula for attention is



> AttentionScore = ((K.T @ Q)/root(head_size))@V








Notes:

    Attention is a communication mechanism. Can be seen as nodes in a directed graph looking at each other and aggregating information with a weighted sum from all nodes that point to them, with data-dependent weights.
    There is no notion of space. Attention simply acts over a set of vectors. This is why we need to positionally encode tokens.

    Each example across batch dimension is of course processed completely independently and never "talk" to each other
    In an "encoder" attention block just delete the single line that does masking with tril, allowing all tokens to communicate. This block here is called a "decoder" attention block because it has triangular masking, and is usually used in autoregressive settings, like language modeling.

    "self-attention" just means that the keys and values are produced from the same source as queries. In "cross-attention", the queries still get produced from x, but the keys and values come from some other, external source (e.g. an encoder module)
    
    "Scaled" attention additional divides wei by 1/sqrt(head_size). This makes it so when input Q,K are unit variance, wei will be unit variance too and Softmax will stay diffuse and not saturate too much. Illustration below



In [ ]:
torch.manual_seed(1337)
B,T,C = 4,8,2 #batch,time,channels
x = torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 2])

In [ ]:
#we want x[b,t] = mean(i<=t)
#version 1
B,T,C = 4,8,2
xbow = torch.zeros((B,T,C))
for b in range(B):
  for t in range(T):
    xprev = x[b,:t+1]
    xbow[b,t] = torch.mean(xprev,0)

xbow[1]

In [ ]:
#version 2
wei = torch.tril(torch.ones(T,T)) #size is (T,T) for each token mean stored in each row till the token t
wei = wei/wei.sum(1,keepdim=True)
xbow2 = wei @ x
torch.allclose(xbow,xbow2,atol=1e-6, rtol=1e-4)
# xbow2[1]

In [ ]:
#version 3
tril = torch.tril(torch.ones(T,T))
wei = torch.zeros((T,T))
wei = wei.masked_fill(tril==0,float('-inf'))
wei = F.softmax(wei,dim=-1)
xbow3 = wei @ x
torch.allclose(xbow,xbow3,atol=1e-6, rtol=1e-4)

In [ ]:
#version 4 self-attention
torch.manual_seed(1337)
B,T,C = 4,8,32 #batch,time,channels
x = torch.randn(B,T,C)

#head self-attention
head_size = 16
key = nn.Linear(C,head_size,bias=False)
query = nn.Linear(C,head_size,bias=False)
value = nn.Linear(C,head_size,bias=False)
k = key(x)
q = query(x)
wei = q @ k.transpose(-2,-1) * C**-0.5

tril = torch.tril(torch.ones(T,T))
# wei = torch.zeros((T,T))
wei = wei.masked_fill(tril==0,float('-inf'))
wei = F.softmax(wei,dim=-1)
v = value(x)
out = wei @ v

out.shape
# torch.allclose(xbow,xbow3,atol=1e-6, rtol=1e-4)

In [ ]:
wei[0]